# Tuto utilisation ARIO3

## Installation


> `git clone https://github.com/spjuhel/ARIO3.git`

## Dépendances :

J'utilise python 3.8 et pas mal de paquets tiers,

Normalement si tu as conda tu peux installer les dépendances facilement avec :

> `conda env create -f environment.yml`

Si tout marche bien il ne reste qu'à activer l'environnement 'ario3' avant d'utiliser le programme ; sinon il faudra installer les dépendances manquantes plus manuellement.

## Utilisation

### Structure de dossiers
<a id='file_structure'></a>

Avant d'expliquer comment paramétrer le modèle et produire les entrées, voilà la structure conseillée pour utiliser le modèle. Le dossier `ARIO3-inputs` contiendra les entrées du modèle (avec certains préparatifs dans `other`), le dossier `ARIO3-outputs` récoltera les sorties du programme de manière générale (et `results` les sorties de simulation).

Tout ceci est paramétrable donc une structure différente peut être choisie, mais il faut faire attention à spécifier les bon paramètres dans ce cas.

``` 
Project
│   
│
└───ARIO3-inputs
│   │   mrio.pkl
│   │   params.json
│   │   mrio_params.json
│   │   event_template.json
│   │
│   └───other
│       │   aggregation_table.ods
│       │   IOT_ixi.zip
│       └   ...
|
└───ARIO3-outputs
│   └───indicators-timestamp.csv
│   │
│   └───logs
│   │   └───run.log [*]
│   │
│   └───results
|       └───run1 [*]
│           └─── ...
│   
└───ARIO3 (from git depot)
    │   ...
    └───other
        │  
        │
        └ 
```

### Génération des entrées

Pour utiliser le programme, la première chose à faire c'est créer les tables IO au bon format, malheureusement je n'ai pas trouvé de moyen de rendre portable celle que je génère donc il faut le faire en local. Mais j'ai fait un script pour rendre ça plus facile. 

Le module pymrio de K. Stadler permet de télécharger automatiquement les tables exiobase avec la commande ci dessous ; je recommande de conserver ces données dans un dossier différent du dossier du code (voir [Structure de dossier](#file_structure))

In [ ]:
import pymrio as pym

exio3_folder = "ARIO3-inputs/other"
exio_meta = pym.download_exiobase3(
    storage_folder=exio3_folder, system="ixi", years=[2011]
)

Une fois la table téléchargée, il faut éventuellement l'aggréger et la mettre au format pickle.
Le script `other/aggreg_exio3.py` doit permettre cela facilement. Pour l'utiliser il faut :
   1. l'archive zip de table exio préalablement téléchargée (par exemple `IOT_2019_ixi.zip`
   2. une matrice de concordance pour aggréger les secteurs définie dans une feuille nommé `input` d'un tableur.
   3. un fichier json de renommage de secteur 
   4. \[optionnellement\] un fichier json d'aggrégation de région
   
2. Le tableur `other/aggreg EXIO.ods` donne un template de matrice de concordance. Les colonnes correspondent à la classification d'origine d'exio3 et les lignes aux aggrégats. Les modifications faites à la première feuille sont automatiquement exportés dans la feuille `input`. Le template peut être utilisé pour aggrégé à 74 secteurs.

3. L'aggrégation renomme les secteurs `sec[0-n]` par défaut. Il faut donc fournir un fichier de renommage. Le fichier `other/sector_renaming74.json` peut être utilisé avec l'aggrégation fournie ou modifié pour une autre aggrégation.

4. Un autre dictionnaire json peut optionnelement être fourni pour aggréger les régions (voir [ici](https://pymrio.readthedocs.io/en/latest/notebooks/aggregation_examples.html#Regional-aggregation-using-the-country-converter-coco)  ainsi que l'exemple `other/japan_aggreg.json`)

Avec ces entrée le script peut être lancé ainsi depuis le répertoire principal d'ARIO3 :

```shell
$ python scripts/aggreg_exio3.py -o "../ARIO3-inputs/mrio" "../ARIO-inputs/IOT_2019_ixi.zip" "./other/aggreg EXIO.ods" "./other/sector_renaming_74.json"

12:25:50 [INFO ] aggreg_axio3 Parsing exiobase3 from [..]/ARIO-inputs/IOT_2019_ixi.zip
12:26:54 [INFO ] aggreg_axio3 Done
12:26:54 [INFO ] aggreg_axio3 Computing the IO components
12:27:48 [INFO ] aggreg_axio3 Done
12:27:48 [INFO ] aggreg_axio3 Reading aggregation matrix from sheet 'input' in file ./other/aggreg EXIO.ods
12:27:49 [INFO ] aggreg_axio3 Aggregating from 163 to 74 sectors
12:28:09 [INFO ] aggreg_axio3 Done
12:28:09 [INFO ] aggreg_axio3 Renaming sectors from ./other/sector_renaming_74.json
12:28:09 [INFO ] aggreg_axio3 Done
12:28:12 [INFO ] aggreg_axio3 Saving to [..]/ARIO3-inputs/mrio_20220216-122812.pkl
```

Ou pour aggreger aussi les régions :

```shell
$ python scripts/aggreg_exio3.py -o "../ARIO3-inputs/mrio" "../ARIO-inputs/IOT_2019_ixi.zip" "./other/aggreg EXIO.ods" "./other/sector_renaming_74.json" "./other/japan_aggreg.json"
```

Ceci produit un fichier pickle `mrio_TIMESTAMP.pkl` (malheureusement difficilement exportable vers un autre système) qui peut être utilisé comme entrée par ARIO3 en le renommant simplement `mrio.pkl`

### Paramètrage du modèle

Il y a deux fichiers de paramétrage à fournir au modèle :

1. Paramétrage de la simulation 

Un fichier json de la forme :

```
{
"storage_dir": "[..]/ARIO3-outputs/",   # Le dossier général des données du modèle
"results_storage": "results",          # Le sous-dossier ou mettre les résultats
"register_stocks": true,               # Garder l'évolution des stocks (bcp d'espace)
"psi_param": 0.85,                     # Le paramètre psi
"model_time_step": 1,                  # La durée en jour d'un pas de temps (pas modifiable)
"timestep_dividing_factor": 365,       # Le nombre de pas de temps par an (pas modifiable)
"inventory_restoration_time": 6,       # Le taux de restauration des inventaire
"alpha_base": 1.0,                     # Le taux de surproduction de base
"alpha_max": 1.25,                     # Le taux de surproduction de maximum
"alpha_tau": 365,                      # Le temps caractéristique de surproduction
"rebuild_tau": 10000,                  # Paramètre de temps de reconstruction (pas modifiable)
"n_timesteps": 1730,                   # Le nombre de pas maximum à simuler
"min_duration": 25,                    # Le nombre minimum de pas à simuler

# La part initial de production allouée à la reconstruction dans le pays impacté
"impacted_region_base_production_toward_rebuilding": 0.001, 
# La part initial de production allouée à la reconstruction dans le reste du monde
"row_base_production_toward_rebuilding": 0.0,

# Le chemin vers les paramètres de la table IO
"mrio_params_file": "ARIO3-inputs/mrio_params.json" # Mettre le chemin absolu
}
```

2. Paramètres de la table IO

Un fichier json de la forme :

```
{  
"monetary_unit": 1000000,             # Le facteur monétaire des table (EXIO3 est en millions)
"capital_ratio_dict": {
    "Nom du secteur 1": x1,           # x1 étant le ratio estimé de Capital/VA du secteur
    ...
    "Nom du secteur n" : x2,
    },
"inventories_dict":{
    # y1 étant le nombre de jours d'inventaire stocké pour ce type de produit
    # ou "inf" si l'on considère un stock infini.
    "Nom du secteur 1": y1,           
    ...
    "Nom du secteur n" : y2,
    }
}
```

Les fichiers mis à disposition dans `other/` peuvent être utilisé comme template pour l'aggrégation 74 secteurs.

### Evénènements

Il faut enfin fournir au modèle un évènement à simuler dans un fichier json au format suivant :

```
{
    # La liste des régions impactés (testé avec une seule pour le moment)
    "aff-regions": [
        "Nom_de_la_région_impactée" (au format ISO_ALPHA_2 pour exiobase)
    ],
    # La liste des secteurs impactés
    "aff-sectors": [
        "Mining of coal and lignite; extraction of peat (10)",
        "Extraction of crude petroleum, gas and other",
        "Mining of uranium and thorium ores (12)",
        "Mining of iron ores",
    ],
    # Comment les dommages sont répartis entre régions (doit correspondre à la liste des régions impactées)
    "dmg-distrib-regions": [
        1
    ],
    # Le type de distribution des dommages sur les secteurs ('gdp' réparti en fonction de l'importance du secteur dans le PIB)
    "dmg-distrib-sectors-type": "gdp",
    # Pour une répartition différente on peut utiliser une liste (comme pour les régions)
    "dmg-distrib-sectors": [],
    # Durée de l'évènement (Non utilisée pour le moment)
    "duration": 1,
    # Nom  de l'évènement (Pas vraiment utilisé)
    "name": "0",
    # Le pas de temps où l'évènement à lieu
    "occur": 5,
    # La quantité unitaire de dommage causés par l'évènement (donc en $/€/£ etc)
    "q_dmg": 50000000000,
    # Un dictionnaire des secteurs assurant la reconstruction et la part des dommages que chacun reconstruit
    "rebuilding-sectors": {
        "Secteur reconstruisant 1" : part_de_dommage_1
        "Secteur reconstruisant 2" : part_de_dommage_2
    }
}
```

Le fichier `other/event_template.json` peut être utilisé/modifié par exemple.

### Lancer une run simple, faire un test

Le script `script/quick_model_test.py` permet de lancer une simulation simple :

```shell
$ python scripts/quick_model_test.py "../ARIO3-inputs/mrio.pkl" "../ARIO3-inputs/params.json" "../ARIO3-inputs/event.json" "../ARIO3-outputs/run1/"
```

Dont voici un exemple d'éxécution :

```shell
You are about to run the ARIO3 model with the following inputs :
        - EXIOBASE 3 pickle file : [..]/ARIO3-inputs/mrio.pkl
        - Model parameters :
            Storage directory : [..]/ARIO3-outputs/run1/
            Steps to simulate : 1730
            MRIO params file  : [..]/ARIO3-inputs/mrio_params.json
        - Event file : [..]/ARIO3-inputs/event.json

Do you wish to proceed ?
     [y/N] y
13:32:54 [INFO] - [simulation.py > __init__() > 57] - Initializing new simulation instance
13:32:55 [DEBUG] - [mriosystem.py > __init__() > 124] - Initiating new MrioSystem instance
13:32:55 [INFO] - [mriosystem.py > __init__() > 142] - Monetary unit is: 1000000
13:32:55 [INFO] - [simulation.py > __init__() > 129] - Initialized !
13:32:55 [INFO] - [simulation.py > read_events_from_list() > 275] - Reading events from given list and adding them to the model
13:32:55 [INFO] - [simulation.py > loop() > 146] - Starting model loop for at most 365 steps
13:32:55 [INFO] - [simulation.py > loop() > 147] - One step is 1 day(s)
13:32:55 [INFO] - [simulation.py > loop() > 152] - Parameters : 
 {
    "results_storage": "results",
    "register_stocks": true,
    "psi_param": 0.85,
    "model_time_step": 1,
    "timestep_dividing_factor": 365,
    "inventory_restoration_time": 6,
    "alpha_base": 1.0,
    "alpha_max": 1.25,
    "alpha_tau": 365,
    "rebuild_tau": 10000,
    "n_timesteps": 1730,
    "min_duration": 25,
    "impacted_region_base_production_toward_rebuilding": 0.001,
    "row_base_production_toward_rebuilding": 0.0,
    "mrio_params_file": "/home/sjuhel/Nextcloud/Thesis/Workbench/Data/BigScaleTest/mrio_params_2.json",
    "storage_dir": "../../Data/BigScaleTest/Japan/"
}
Processed: Step: 0  ~ N/A% ETA:  --:--:--
13:32:55 [INFO] - [simulation.py > shock() > 309] - Shocking model with new event
13:32:55 [INFO] - [simulation.py > shock() > 310] - Affected regions are : ['Japan']
13:32:55 [INFO] - [simulation.py > shock() > 325] - Damages are 50000.0 times 1000000 [unit (ie $/€/£)]
Processed: Step: 66  ~   3% ETA:   0:00:14
13:32:56 [INFO] - [mriosystem.py > calc_production() > 279] - At least one industry entered shortage regime
Processed: Step: 489  ~  28% ETA:   0:00:07
13:32:58 [INFO] - [mriosystem.py > calc_production() > 295] - All industries exited shortage regime
Processed: Step: 768  ~  44% ETA:   0:00:05
13:33:00 [WARNING] - [simulation.py > loop() > 170] - Economy seems to have found an equilibrium
                    - At step : 781
                    
Processed: Step: 1730  ~ 100% Time:  0:00:04                                                              
13:33:00 [INFO] - [indicators.py > __init__() > 30] - Instanciating indicators
13:33:03 [INFO] - [indicators.py > save_dfs() > 331] - Saving computed dataframe to results folder
13:33:08 [INFO] - [indicators.py > update_indicators() > 315] - (Re)computing all indicators

{ print output of indicators }

```


## Sorties et indicateurs

### Sorties du modèle

L'exécution du modèle produit les fichiers binaires suivants, qui sont des enregistrement au cours de la simulation des différentes variable. De manière générale, une ligne correspond à un pas de temps.

Les enregistrements sont des arrays numpy pouvant être lu avec :

```python
numpy.memmap("chemin du fichier", mode='r+', dtype='float64', shape = array_shape)
```

   * classic_demand_record : la demande intermédiaire ; array_shape = (n_steps, n_regions * n_sectors)
   * final_demand_unmet_record : la demande final non remplie ; array_shape = (n_steps, n_regions * n_sectors)
   * iotable_X_max_record : la capacité de production ; array_shape = (n_steps, n_regions * n_sectors)
   * iotable_XVA_record : la production réalisée ; array_shape = (n_steps, n_regions * n_sectors)
   * limiting_stocks_record : les inputs limitant ; array_shape = (n_steps * n_sectors, n_regions * n_sectors) (attention le dtype est bool ici)
   * overprodvector_record : le vecteur de surproduction ; array_shape = (n_steps, n_regions * n_sectors)
   * rebuild_demand_record : la demande pour la reconstruction ; array_shape = (n_steps, n_regions * n_sectors)
   * rebuild_prod_record : la production allouée pour la reconstruction ; array_shape = (n_steps, n_regions * n_sectors)
   * stocks_record : les 'inventaires' de chaque inputs ; n_steps * n_sectors, n_regions * n_sectors) (L'enregistrement étant optionnel dans les paramètres, le fichier peut ne pas exister)

Ainsi que :

   * simulation.log : Les logs de la simulation
   * simulated_events.json : L'évènement simulé
   * simulated_params.json : Les paramètre utilisé (Utiles lorsque l'on fait plein de simulation différentes)

Afin de s'éviter des traitements de fichiers pénibles ; la classe `Indicators` peut être utilisée. C'est d'ailleurs le cas dans le script `quick_model_test.py`.

En supposant que `params` est un dictionnaire des paramètres de simulation :

```python
indic = Indicators.from_storage_path(pathlib.Path(params['storage_dir']), params=params)
```

Cette commande a deux effets :

- l'instanciation de l'objet `indic` (cf documentation)
- la création des fichiers `treated_df.feather` et `treated_df_loss.feather`, ainsi que du dossier `treated_df_stocks.parquet` (si l'enregistrement des stocks a été demandé lors de la simulation).

Ces fichiers sont des dataframes regroupant l'ensemble des variables et peuvent être lus ainsi :

```python
df = pandas.read_feather("treated_df.feather")
df_stocks = pandas.read_parquet("treated_df_stocks.parquet/")
```

In [56]:
df

,region,sector,step,variable,value
0,AT,Air transport (62),0,production,18.318554
1,AT,Air transport (62),1,production,18.318554
2,AT,Air transport (62),2,production,18.318554
3,AT,Air transport (62),3,production,18.318554
4,AT,Air transport (62),4,production,18.318554
...,...,...,...,...,...
8745907,ZA,"Wholesale trade and commission trade, except o...",397,overprod,1.000005
8745908,ZA,"Wholesale trade and commission trade, except o...",398,overprod,1.000005
8745909,ZA,"Wholesale trade and commission trade, except o...",399,overprod,1.000005
8745910,ZA,"Wholesale trade and commission trade, except o...",400,overprod,1.000005
